In [1]:
import numpy as np
from qutip import *
import matplotlib.pyplot as plt
from visualisation_tools import *
from model_generator import *
from qsq_protocol import *
from csv_utils import *
from optimizer_utils import *
plt.rcParams.update({
    "figure.figsize": (10, 6),
    "text.usetex": False, # Remove this line if your code doesn't run/ take too long
    "font.family": 'Times New Roman',
    "figure.dpi": 100,
    "font.size": 14
})
import os
import csv

In [5]:
#  Define the CSV file path
csv_file = 'data2.csv'
# generate_model_data(csv_file, 'uniform',2000)
# for strength in np.linspace(0.3,0.5,100):
#     generate_model_data(csv_file, 'perturbed',30,perturbation=strength)
generate_model_data(csv_file, 'random',10)

# Usage
data = load_model_data(csv_file)

# # Access the data by variable name, for example:
# print(data['models'])     # Access the models (now Qobjs)



In [6]:
fill_model_gauge_fidelity("data2.csv",10)

10 rows to be processed
[10/10] flushed (100.0%)


In [7]:
from csv_utils import _rewrite_csv

In [ ]:
def generate_unitary_gauge_model(csv_file, flush_every=10,simple=False):
    # Step 1: read everything
    with open(csv_file, mode='r', newline='') as file:
        reader = csv.DictReader(file)
        fieldnames = reader.fieldnames
        rows = list(reader)

    print(f"{len(rows)} rows to be processed")
    # Step 2: process rows in place
    for i, row in enumerate(rows, start=1):
        if not row.get('model_average_fidelity_gauge'):
            model_data = eval(row['model'])
            rho, gate, M = [Qobj(m) for m in model_data]
            avg_fidelity = float(row['model_average_fidelity'])

            row['model_average_fidelity_gauge'] = average_fidelity_gauge(
                rho, gate, M, avg_fidelity
            )

        # Flush FULL dataset
        if i % flush_every == 0:
            print(f"[{i}/{len(rows)}] flushed ({100*i/len(rows):.1f}%)")
            _rewrite_csv(csv_file, fieldnames, rows)
    

    # Final flush
    _rewrite_csv(csv_file, fieldnames, rows)